# PostgreSQL Advanced

# Triggers in PostgreSQL

Een trigger in PostgreSQL is een mechanisme waarmee je een specifieke actie automatisch kunt uitvoeren als reactie op een bepaalde gebeurtenis in de database, zoals een `INSERT`, `UPDATE` of `DELETE` op een tabel. Triggers kunnen worden gebruikt om logica af te dwingen, auditing uit te voeren, gegevensintegriteit te behouden, of complexere acties te automatiseren.

Een trigger bestaat uit:
1. **Trigger-event**: De gebeurtenis die de trigger activeert, zoals een wijziging in een rij.
2. **Trigger-actie**: De actie die wordt uitgevoerd (via een functie).
3. **Timing**: Wanneer de trigger wordt uitgevoerd: vóór (`BEFORE`) of na (`AFTER`) het event.

## Voorbeelden

### **Voorbeeld 1: Bijhouden van wijzigingen in de tabel `countries`**
Als een record in de tabel `countries` wordt geüpdatet, willen we de oude gegevens opslaan in een logtabel `countries_log`.

#### Stap 1: Logtabel maken
```sql
CREATE TABLE countries_log (
    id SERIAL PRIMARY KEY,
    code CHAR(3),
    name VARCHAR(64),
    changed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

#### Stap 2: Triggerfunctie schrijven
Een functie in PostgreSQL bepaalt **wat de trigger doet**.
```sql
CREATE OR REPLACE FUNCTION log_country_update()
RETURNS TRIGGER AS $$
BEGIN
    INSERT INTO countries_log (code, name, changed_at)
    VALUES (OLD.code, OLD.name, CURRENT_TIMESTAMP);
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;
```

#### Stap 3: Trigger aanmaken
De trigger koppelen aan de `countries`-tabel.
```sql
CREATE TRIGGER trigger_log_country_update
AFTER UPDATE ON countries
FOR EACH ROW
EXECUTE FUNCTION log_country_update();
```

Nu wordt bij elke wijziging in de tabel `countries` een log bijgehouden in `countries_log`.

### **Voorbeeld 2: Budgetcontrole op films**
Als iemand een nieuw record toevoegt in de tabel `films` en het budget hoger is dan een bepaald bedrag (bijvoorbeeld 10 miljoen), willen we een foutmelding genereren.

#### Stap 1: Triggerfunctie schrijven
```sql
CREATE OR REPLACE FUNCTION check_film_budget()
RETURNS TRIGGER AS $$
BEGIN
    IF NEW.budget > 10000000 THEN
        RAISE EXCEPTION 'Budget mag niet hoger zijn dan 10 miljoen.';
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;
```

#### Stap 2: Trigger aanmaken
```sql
CREATE TRIGGER trigger_check_budget
BEFORE INSERT ON films
FOR EACH ROW
EXECUTE FUNCTION check_film_budget();
```

Hiermee wordt elke poging om een film met een te hoog budget toe te voegen, tegengehouden.

### **Voorbeeld 3: Automatisch invullen van gross_savings**
Automatisch bereken en update de kolom `gross_savings` in de tabel `economies` op basis van het `gdp_percapita`.

#### Stap 1: Triggerfunctie schrijven
```sql
CREATE OR REPLACE FUNCTION update_gross_savings()
RETURNS TRIGGER AS $$
BEGIN
    NEW.gross_savings = NEW.gdp_percapita * 0.2; -- Bijv. 20% van GDP
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;
```

#### Stap 2: Trigger aanmaken
```sql
CREATE TRIGGER trigger_update_gross_savings
BEFORE UPDATE ON economies
FOR EACH ROW
EXECUTE FUNCTION update_gross_savings();
```

Nu wordt `gross_savings` automatisch opnieuw berekend bij elke wijziging in `gdp_percapita`.

## Oefeningen

### **Oefening 1: Logboek bijhouden van updates**
**Doel:** Houd bij elke update in de tabel `cities` bij wat er is gewijzigd.

1. Maak een tabel `cities_log` met de volgende kolommen:
   - `id` (SERIAL, primaire sleutel)
   - `city_name` (oude naam van de stad)
   - `updated_at` (timestamp van de update)

2. Schrijf een triggerfunctie die bij elke update de oude waarde van de `name` kolom in `cities` opslaat in de logtabel.

3. Koppel een trigger aan de tabel `cities`, zodat deze de functie uitvoert bij elke update.

### **Oefening 2: Budgetcontrole op films**
**Doel:** Zorg ervoor dat geen film in de tabel `films` een budget van meer dan 20 miljoen mag hebben.

1. Schrijf een triggerfunctie die controleert of het budget (`budget`) van een film groter is dan 20 miljoen.
2. Als het budget te hoog is, genereer een foutmelding (`RAISE EXCEPTION`).
3. Maak een trigger die de functie aanroept vóór elke `INSERT` in de tabel `films`.

### **Oefening 3: Automatisch invullen van gross_savings**
**Doel:** Automatisch bereken en update de kolom `gross_savings` in de tabel `economies` op basis van het `gdp_percapita`.

1. Schrijf een triggerfunctie die de waarde van `gross_savings` instelt als 15% van `gdp_percapita`.
2. Zorg ervoor dat de functie wordt uitgevoerd vóór elke `INSERT` of `UPDATE` op de tabel `economies`.

### **Oefening 4: Controle op ontbrekende relaties**
**Doel:** Zorg ervoor dat elke film in de tabel `films` een geldig land in de tabel `countries` heeft.

1. Schrijf een triggerfunctie die controleert of de waarde in de kolom `country` van `films` overeenkomt met een bestaande `code` in de tabel `countries`.
2. Als er geen overeenkomst is, genereer een foutmelding (`RAISE EXCEPTION`).
3. Maak een trigger die de functie aanroept vóór elke `INSERT` in de tabel `films`.

### **Oefening 5: Rollen loggen**
**Doel:** Houd een logboek bij van elke nieuwe rol die aan een persoon wordt toegevoegd in de tabel `roles`.

1. Maak een tabel `roles_log` met de volgende kolommen:
   - `id` (SERIAL, primaire sleutel)
   - `person_id` (ID van de persoon)
   - `film_id` (ID van de film)
   - `role` (toegevoegde rol)
   - `added_at` (timestamp)

2. Schrijf een triggerfunctie die een nieuwe rij toevoegt aan `roles_log` telkens wanneer een nieuwe rol in de tabel `roles` wordt ingevoerd.
3. Koppel een trigger aan de tabel `roles` die de functie uitvoert bij elke `INSERT`.

### **Oefening 6: Update verificatie**
**Doel:** Zorg ervoor dat de `income_group` in de tabel `economies2015` niet wordt gewijzigd als deze al een waarde bevat.

1. Schrijf een triggerfunctie die controleert of de kolom `income_group` al een waarde bevat.
2. Als er al een waarde is, genereer een foutmelding (`RAISE EXCEPTION`).
3. Maak een trigger die de functie aanroept vóór elke `UPDATE` op de tabel `economies2015`.

### **Oefening 7: Facebook likes bijwerken**
**Doel:** Bereken en update automatisch de IMDb-score op basis van het aantal Facebook-likes en stemmen in de tabel `reviews`.

1. Voeg een nieuwe kolom `calculated_score` toe aan de tabel `reviews`.
2. Schrijf een triggerfunctie die `calculated_score` instelt op basis van de formule:
   ```
   calculated_score = (facebook_likes / 1000) + (num_votes / 100) + imdb_score
   ```
3. Zorg ervoor dat deze functie wordt uitgevoerd vóór elke `INSERT` of `UPDATE` op de tabel `reviews`.